In [1]:
%load_ext autoreload
%autoreload 2

import manify
from manify.manifolds import Manifold
import torch
import geoopt

In [ ]:
for curv, dim in [(-1, 2), (0, 2), (1, 2), (-1, 64), (0, 64), (1, 64)]:
    M = Manifold(curvature=curv, dim=dim)

    # Does device switching work?
    M.to("cpu")

    # Do attributes work correctly?
    if curv < 0:
        assert M.type == "H" and isinstance(M.manifold.base, geoopt.Lorentz)
    elif curv == 0:
        assert M.type == "E" and isinstance(M.manifold.base, geoopt.Euclidean)
    else:
        assert M.type == "S" and isinstance(M.manifold.base, geoopt.Sphere)

    # get some vectors via gaussian mixture
    cov = torch.eye(M.dim) / M.dim / 100
    means = torch.vstack([M.mu0] * 10)
    covs = torch.stack([cov] * 10)
    X1, _ = M.sample(z_mean=means, sigma=covs)
    X2, _ = M.sample(z_mean=means[:5], sigma=covs[:5])

    # Verify points are on manifold
    assert M.manifold.check_point(X1), "X1 is not on the manifold"
    assert M.manifold.check_point(X2), "X2 is not on the manifold"

    # Inner products
    ip_11 = M.inner(X1, X1)
    assert ip_11.shape == (10, 10), "Inner product shape mismatch for X1"
    ip_12 = M.inner(X1, X2)
    assert ip_12.shape == (10, 5), "Inner product shape mismatch for X1 and X2"
    if curv == 0:
        assert torch.allclose(ip_11, X1 @ X1.T), "Euclidean inner products do not match for X1"
        assert torch.allclose(ip_12, X1 @ X2.T), "Euclidean inner products do not match for X1 and X2"

    # Dists
    dists_11 = M.dist(X1, X1)
    assert dists_11.shape == (10, 10), "Distance shape mismatch for X1"
    dists_12 = M.dist(X1, X2)
    assert dists_12.shape == (10, 5), "Distance shape mismatch for X1 and X2"
    if curv == 0:
        assert torch.allclose(
            dists_12, torch.linalg.norm(X1[:, None] - X2[None, :], dim=-1)
        ), "Euclidean distances do not match for X1 and X2"
        assert torch.allclose(
            dists_11, torch.linalg.norm(X1[:, None] - X1[None, :], dim=-1)
        ), "Euclidean distances do not match for X1"
    assert (dists_11.triu(1) >= 0).all(), "Distances for X1 should be non-negative"
    assert (dists_12.triu(1) >= 0).all(), "Distances for X2 should be non-negative"
    assert torch.allclose(dists_11.triu(1), M.pdist(X1).triu(1)), "dist and pdist diverge for X1"

    # Square dists
    sqdists_11 = M.dist2(X1, X1)
    assert sqdists_11.shape == (10, 10), "Squared distance shape mismatch for X1"
    sqdists_12 = M.dist2(X1, X2)
    assert sqdists_12.shape == (10, 5), "Squared distance shape mismatch for X1 and X2"
    if curv == 0:
        assert torch.allclose(
            sqdists_12, torch.linalg.norm(X1[:, None] - X2[None, :], dim=-1) ** 2
        ), "Euclidean squared distances do not match for X1 and X2"
        assert torch.allclose(
            sqdists_11, torch.linalg.norm(X1[:, None] - X1[None, :], dim=-1) ** 2
        ), "Euclidean squared distances do not match for X1"
    assert (sqdists_11.triu(1) >= 0).all(), "Squared distances for X1 should be non-negative"
    assert (sqdists_12.triu(1) >= 0).all(), "Squared distances for X1 and X2 should be non-negative"
    assert torch.allclose(sqdists_11.triu(1), M.pdist2(X1).triu(1)), "sqdists_11 and pdist2 diverge for X1"

    # Log-likelihood
    lls = M.log_likelihood(X1)
    if curv == 0:
        # Evaluate as ll of gaussian with mean 0, variance 1:
        assert torch.allclose(
            lls,
            -0.5 * (torch.sum(X1**2, dim=-1) + X1.size(-1) * math.log(2 * math.pi)),
        ), "Log-likelihood mismatch for Gaussian"
    assert (lls <= 0).all(), "Log-likelihood should be non-positive"

    # Logmap and expmap
    logmap_x1 = M.logmap(X1)
    assert M.manifold.check_vector(logmap_x1), "Logmap point should be in the tangent plane"
    expmap_x1 = M.expmap(logmap_x1)
    assert M.manifold.check_point(expmap_x1), "Expmap point should be on the manifold"
    assert torch.allclose(expmap_x1, X1, atol=1e-5), "Expmap does not return the original points"

    # Stereographic conversions
    M_stereo, X1_stereo, X2_stereo = M.stereographic(X1, X2)
    assert M_stereo.is_stereographic
    X_inv_stereo, X1_inv_stereo, X2_inv_stereo = M_stereo.inverse_stereographic(X1_stereo, X2_stereo)
    assert not X_inv_stereo.is_stereographic
    assert torch.allclose(X1_inv_stereo, X1), "Inverse stereographic conversion mismatch for X1"
    assert torch.allclose(X2_inv_stereo, X2), "Inverse stereographic conversion mismatch for X2"

    # Apply
    @M.apply
    def apply_function(x):
        return torch.nn.functional.relu(x)

    result = apply_function(X1)
    assert result.shape == X1.shape, "Result shape mismatch for apply_function"
    assert M.manifold.check_point(result)

In [72]:
expmap_x1

tensor([[ 1.0011,  0.0458,  0.0055],
        [ 1.0001, -0.0142, -0.0087],
        [ 1.0121,  0.1557,  0.0073],
        [ 1.0099, -0.0979,  0.1019],
        [ 1.0033,  0.0339,  0.0737],
        [ 1.0008,  0.0300,  0.0255],
        [ 1.0006,  0.0211,  0.0289],
        [ 1.0040, -0.0701, -0.0553],
        [ 1.0160,  0.1332, -0.1208],
        [ 1.0026,  0.0174,  0.0700]], grad_fn=<CatBackward0>)

In [73]:
X1

tensor([[ 1.0011,  0.0458,  0.0055],
        [ 1.0001, -0.0142, -0.0087],
        [ 1.0121,  0.1557,  0.0073],
        [ 1.0099, -0.0979,  0.1019],
        [ 1.0033,  0.0339,  0.0737],
        [ 1.0008,  0.0300,  0.0255],
        [ 1.0006,  0.0211,  0.0289],
        [ 1.0040, -0.0701, -0.0553],
        [ 1.0160,  0.1332, -0.1208],
        [ 1.0026,  0.0174,  0.0700]], grad_fn=<CatBackward0>)

In [59]:
# make a stack of (10, 2, 2) from this
my_stack = torch.stack([cov] * 10, dim=0)  # create a stack of 10 copies of cov

In [66]:
torch.stack([M.mu0] * 10, dim=1).shape

torch.Size([1, 10, 3])

In [67]:
M.mu0

tensor([[1., 0., 0.]])

In [70]:
torch.vstack([M.mu0] * 10)

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [ ]:
from manify.manifolds import ProductManifold

signature = [(-1, 2), (0, 2), (1, 2)]

pm = ProductManifold(signature=signature)

# get some vectors via gaussian mixture
cov = torch.eye(pm.dim) / pm.dim / 100
means = torch.vstack([pm.mu0] * 10)
covs = torch.stack([cov] * 10)
sigma_factorized = pm.factorize(covs)
torch.random.manual_seed(42)
X1, _ = M.sample(z_mean=means, sigma_factorized=sigma_factorized)
X2, _ = M.sample(z_mean=means[:5], sigma=sigma_factorized[:5])

IndexError: index 6 is out of bounds for dimension 0 with size 6

In [79]:
pm.factorize(covs.flatten())

[tensor([0.0017, 0.0000, 0.0000]),
 tensor([0., 0.]),
 tensor([0.0000, 0.0000, 0.0017])]

In [ ]:
covs.reshape(10, -1)

tensor([[0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017],
        [0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017],
        [0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0017],
        [0.0017, 0.0000, 

In [ ]:
for signature in [[(-1, 8)], [(0, 8)], [(1, 8)], [(-1, 8), (1, 8)], [(-1, 8), (0, 8), (1, 8)], [(0, 8), (0, 8)]]:
    pm = ProductManifold(signature=signature)

    # get some vectors via gaussian mixture
    covs = [torch.stack([torch.eye(M.dim) / M.dim / 100] * 10) for M in pm.P]
    means = torch.vstack([pm.mu0] * 10)
    torch.random.manual_seed(42)
    X1, _ = pm.sample(z_mean=means, sigma_factorized=covs)
    X2, _ = pm.sample(z_mean=means[:5], sigma_factorized=[cov[:5] for cov in covs])

    # Do attributes work correctly?
    for M in pm.P:
        curv = M.curvature
        if curv < 0:
            assert M.type == "H" and isinstance(M.manifold.base, geoopt.Lorentz)
        elif curv == 0:
            assert M.type == "E" and isinstance(M.manifold.base, geoopt.Euclidean)
        else:
            assert M.type == "S" and isinstance(M.manifold.base, geoopt.Sphere)

    # _shared_tests(pm, X1, X2, is_euclidean=all(M.curvature == 0 for M in pm.P))
    print(all(M.curvature == 0 for M in pm.P))

False
True
False
False
False
True


In [114]:
pm = ProductManifold(signature=[(0, 8), (0, 8), (0, 8)])

X, y = pm.gaussian_mixture()
pm_stereo, X_stereo = pm.stereographic(X)
_, X2 = pm_stereo.inverse_stereographic(X_stereo)

In [ ]:
import torch

from sklearn.model_selection import train_test_split

from manify.predictors.decision_tree import ProductSpaceDT, ProductSpaceRF
from manify.predictors.kappa_gcn import KappaGCN
from manify.predictors.perceptron import ProductSpacePerceptron
from manify.predictors.svm import ProductSpaceSVM
from manify.manifolds import ProductManifold


print("Testing basic classifier functionality")
pm = ProductManifold(signature=[(-1, 2), (0, 2), (1, 2)])
X, y = pm.gaussian_mixture(num_points=100, num_classes=2, seed=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Init models
for model_class, args in [
    # ProductSpaceDT,
    # ProductSpaceRF,
    (KappaGCN, {"output_dim": 2, "hidden_dims": [pm.dim, pm.dim]}),
    (ProductSpacePerceptron, {}),
    (ProductSpaceSVM, {}),
]:
    print(f"  Testing class: {model_class.__name__} ")
    model = model_class(pm=pm, **args)
    model.fit(X_train, y_train)

    # Predictions
    preds = model.predict(X_test)
    assert preds.shape[0] == X_test.shape[0], "Predictions should match the number of test samples"
    assert preds.ndim == 1, "Predictions should be a 1D array"

    # Probabilities
    probs = model.predict_proba(X_test)
    assert probs.shape == (X_test.shape[0], 2)
    assert probs.ndim == 2, "Probabilities should be a 2D array"
    assert torch.argmax(probs, dim=1) == preds

    # Accuracies
    accuracy = model.score(X_test, y_test)
    assert torch.isclose(accuracy, (preds == y_test).float().mean(), atol=1e-5), "Accuracy calculation mismatch"
    assert accuracy >= 0.5, f"Model {model_class.__name__} did not achieve sufficient accuracy"

Testing basic classifier functionality
  Testing class: KappaGCN 


ValueError: ProductManifold must be stereographic for KappaGCN to work. Please use pm.stereographic() to convert.

In [2]:
from manify.predictors.kappa_gcn import get_A_hat

print("Testing basic classifier functionality")
pm = ProductManifold(signature=[(-1, 2), (0, 2), (1, 2)])
X, y = pm.gaussian_mixture(num_points=100, num_classes=2, seed=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

pm_stereo, X_train_stereo, X_test_stereo = pm.stereographic(X_train, X_test)
kappa_gcn = KappaGCN(pm=pm_stereo, output_dim=2, hidden_dims=[pm.dim, pm.dim])


X_train_kernel = torch.exp(-pm.pdist2(X_train))
X_test_kernel = torch.exp(-pm.pdist2(X_test))
A_train = get_A_hat(X_train_kernel)
A_test = get_A_hat(X_test_kernel)
kappa_gcn.fit(X_train, y_train, A=A_train, use_tqdm=False, epochs=100)

Testing basic classifier functionality


NameError: name 'ProductManifold' is not defined

In [1]:
# Test decision tree
import manify
from sklearn.model_selection import train_test_split

pm = manify.manifolds.ProductManifold(signature=[(-1, 2), (0, 2), (1, 2)])

X, y = pm.gaussian_mixture(num_points=100, num_classes=2, seed=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

dt = manify.predictors.decision_tree.ProductSpaceDT(pm=pm, max_depth=3)
dt.fit(X_train, y_train)

In [2]:
import manify.predictors


rf = manify.predictors.decision_tree.ProductSpaceRF(pm=pm, max_depth=3, n_estimators=10)
rf.fit(X_train, y_train)

(rf.predict(X_test) == y_test)

tensor([ True,  True,  True,  True, False,  True, False,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,  True])

In [3]:
perceptron = manify.predictors.perceptron.ProductSpacePerceptron(pm=pm)

perceptron.fit(X_train, y_train)
perceptron.predict(X_test)

tensor([1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [4]:
perceptron.alpha[]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1149281247.py, line 1)

In [11]:
import manify

a, b, c, d = manify.utils.dataloaders.load_hf("cora")

README.md:   0%|          | 0.00/596 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.72M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
for name, x in zip(["features", "dists", "labels", "adjacency"], [a, b, c, d]):
    try:
        print(name, x.shape)
    except Exception:
        print(name, "None")

features None
dists torch.Size([2485, 2485])
labels torch.Size([2485, 2485])
adjacency torch.Size([2485])


In [2]:
import manify

# Load karate club dataset
_, dists, adj, _ = manify.utils.dataloaders.load_hf("karate_club")
pm = manify.manifolds.ProductManifold(signature=[(-1, 2), (0, 2), (1, 2)])

# Run without train_test_split
X, losses = manify.embedders.coordinate_learning.train_coords(
    pm=pm, dists=dists, burn_in_iterations=10, training_iterations=90
)
losses

  0%|          | 0/100 [00:00<?, ?it/s]

{'train_train': [407.38525390625,
  406.9979248046875,
  406.6103515625,
  406.222412109375,
  405.83416748046875,
  405.4454650878906,
  405.05657958984375,
  404.6674499511719,
  404.28057861328125,
  403.893310546875,
  403.50567626953125,
  399.615966796875,
  395.69378662109375,
  391.74444580078125,
  387.99896240234375,
  384.5290832519531,
  381.21240234375,
  377.8980407714844,
  374.60894775390625,
  371.4312438964844,
  368.331298828125,
  365.3991394042969,
  362.5733947753906,
  359.8371887207031,
  357.2330322265625,
  354.6155090332031,
  351.9755859375,
  349.33917236328125,
  346.822021484375,
  344.3118896484375,
  341.82952880859375,
  339.3853759765625,
  336.925537109375,
  334.455810546875,
  331.9468994140625,
  329.4891662597656,
  327.0758972167969,
  324.71466064453125,
  322.3912353515625,
  320.1859436035156,
  317.9987487792969,
  315.8346862792969,
  313.648681640625,
  311.5186767578125,
  309.3787536621094,
  307.2815246582031,
  305.29876708984375,
  30